# Getting TMDB Data for multi-class classification

In [1]:
#Using API
url="https://api.themoviedb.org/3/movie/top_rated?api_key=7ce14097cd9673cf8aa48a9a52eee5f9&language=en-US&page="

In [2]:
total_pages=471

In [3]:
# Getting data of all the pages
import requests
import pandas as pd

In [4]:
data_list=[]

In [5]:

for i in range(1,total_pages+1):
    data=requests.get(url+str(i))
    data_list.extend(data.json()['results'])
    

In [6]:
df=pd.DataFrame(data_list)

In [7]:
df=df[['title','overview','genre_ids']]

In [8]:
df.sample(5)

,title,overview,genre_ids
3042,My Brother Chases Dinosaurs,"As a child, Jack believed the tender lie his p...","[35, 10751]"
6156,Assassination Nation,"High school senior Lily and her friends, Bex, ...","[53, 27, 35]"
2978,The Towering Inferno,At the opening party of a colossal—but poorly ...,"[28, 53, 18]"
9122,Green Lantern,"For centuries, a small but powerful force of w...","[12, 28, 53, 878]"
7099,Welcome Home Roscoe Jenkins,When a celebrated TV show host returns to his ...,"[35, 18]"


In [9]:
genre_data=requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=7ce14097cd9673cf8aa48a9a52eee5f9&language=en-US')

In [10]:
gl=genre_data.json()['genres']

In [11]:
genre_dict={}

In [12]:
#Updating a dictionary of genres
for i in range(len(gl)):
    genre_dict[gl[i]['id']]=gl[i]['name']

In [13]:
#Function to convert genre_ids to genres
def genre_text(glist):
    l=[]
    for i in glist:
       l.append(genre_dict[i])
    return l


In [14]:
df['genre_ids']=df['genre_ids'].apply(genre_text)

In [15]:
df

,title,overview,genre_ids
0,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[Drama, Crime]"
2,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
3,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,[Romance]
4,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
...,...,...,...
9415,Superman IV: The Quest for Peace,With global superpowers engaged in an increasi...,"[Action, Adventure, Science Fiction]"
9416,Movie 43,Fifteen directors and an all-star cast created...,[Comedy]
9417,Jason Goes to Hell: The Final Friday,"Jason Voorhees, the living, breathing essence ...",[Horror]
9418,Day of the Dead: Bloodline,"In a world overrun by zombies, military person...","[Action, Horror]"


## Text- Preprocessing

In [16]:
# 1. lower casing
df['overview']=df['overview'].str.lower()

In [17]:
df.head(2)

,title,overview,genre_ids
0,Dilwale Dulhania Le Jayenge,"raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
1,The Shawshank Redemption,framed in the 1940s for the double murder of h...,"[Drama, Crime]"


In [18]:
df['overview']

0       raj is a rich, carefree, happy-go-lucky second...
1       framed in the 1940s for the double murder of h...
2       spanning the years 1945 to 1955, a chronicle o...
3       professor gabriel emerson finally learns the t...
4       the true story of how businessman oskar schind...
                              ...                        
9415    with global superpowers engaged in an increasi...
9416    fifteen directors and an all-star cast created...
9417    jason voorhees, the living, breathing essence ...
9418    in a world overrun by zombies, military person...
9419    since his triumphant rise to power in the orig...
Name: overview, Length: 9420, dtype: object

In [19]:
# 2. Removinf HTML tag
import re
def remove_html_tag(text):
    pattern=re.compile(r'<.*?>')
    return pattern.sub(r'',text)


In [20]:
sample_text='<div class="devsite-nav-item-title">TensorFlow Certificate program  </div> <div class="devsite-nav-item-description">  entiate yourself by demonstrating your ML proficiency</div>'

In [21]:
remove_html_tag(sample_text)

'TensorFlow Certificate program     entiate yourself by demonstrating your ML proficiency'

In [22]:
# Apllying the remove html function
df['overview']=df['overview'].apply(remove_html_tag)

In [23]:
# for i in range(100):
#     print(i,'*****************************','\n',df['overview'][i])
    

In [24]:
# 4. Remove punctuation
import string
exclude=string.punctuation
print(exclude)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [25]:
def remove_punc(text):
    for char in exclude:
        text=text.replace(char,'')
    return text


In [26]:
df['overview']=df['overview'].apply(remove_punc)

In [27]:
# 6. Spelling Correction
from textblob import TextBlob

In [28]:
incorrect_string='I am not goid to eat in the mornign conditojn when i am feeling hunfry'

In [29]:
tb=TextBlob(incorrect_string)
tb.correct().string

'I am not good to eat in the morning condition when i am feeling hungry'

In [30]:
# 7. Removing StopWords
from nltk.corpus import stopwords

In [31]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x=new_text[:]
    new_text.clear()
    return " ".join(x)


In [32]:
df['overview']=df['overview'].apply(remove_stopwords)

In [33]:
for i in range(100):
    print(i,'*****************************','\n',df['overview'][i])

0 ***************************** 
 raj   rich carefree happygolucky second generation nri simran   daughter  chaudhary baldev singh   spite    nri   strict  adherence  indian values simran  left  india   married   childhood fiancé raj leaves  india   mission   hands  claim  lady love   noses   whole family thus begins  saga
1 ***************************** 
 framed   1940s   double murder   wife   lover upstanding banker andy dufresne begins  new life   shawshank prison   puts  accounting skills  work   amoral warden   long stretch  prison dufresne comes   admired    inmates including  older prisoner named red   integrity  unquenchable sense  hope
2 ***************************** 
 spanning  years 1945  1955  chronicle   fictional italianamerican corleone crime family  organized crime family patriarch vito corleone barely survives  attempt   life  youngest son michael steps   take care   wouldbe killers launching  campaign  bloody revenge
3 ***************************** 
 professor gabrie

In [34]:
# 9. Tokenization
from nltk.tokenize import word_tokenize

In [35]:
df['overview'].apply(word_tokenize)

0       [raj, rich, carefree, happygolucky, second, ge...
1       [framed, 1940s, double, murder, wife, lover, u...
2       [spanning, years, 1945, 1955, chronicle, ficti...
3       [professor, gabriel, emerson, finally, learns,...
4       [true, story, businessman, oskar, schindler, s...
                              ...                        
9415    [global, superpowers, engaged, increasingly, h...
9416    [fifteen, directors, allstar, cast, created, t...
9417    [jason, voorhees, living, breathing, essence, ...
9418    [world, overrun, zombies, military, personnel,...
9419    [since, triumphant, rise, power, original, blo...
Name: overview, Length: 9420, dtype: object

In [36]:
# 10. Stemming 
from nltk.stem.porter import PorterStemmer

In [37]:
ps=PorterStemmer()

In [38]:
def stem_words(text):
    return " ".join(ps.stem(word) for word in text.split())

In [39]:
df['overview']=df['overview'].apply(stem_words)

In [41]:
df['overview'][0]

'raj rich carefre happygolucki second gener nri simran daughter chaudhari baldev singh spite nri strict adher indian valu simran left india marri childhood fiancé raj leav india mission hand claim ladi love nose whole famili thu begin saga'